In [42]:
import os
import pandas as pd
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer
from transformers import BertModel
import torch
import torch.nn as nn


In [ ]:
class BertNerModel(nn.Module):
    def __init__(self,class_num):
        super().__init__()
        self.bert=BertModel.from_pretrained(os.path.join("bert-base-chinese"))
        self.classifier=nn.Linear(768,class_num)

        self.loss=nn.CrossEntropyLoss()
    def forward(self,batch_index,batch_label=None):
        self.bert(batch_index)
        

In [35]:
class BertDataset(Dataset):
    def __init__(self,texts,lablels,label2index,max_len,tokenizer):
        self.texts=texts
        self.labels=lablels
        self.label2index=label2index
        self.tokenizer=tokenizer
        self.max_len=max_len

    def __getitem__(self, index):
        text=self.texts[index]
        label=self.labels[index][:self.max_len]

        text_index=self.tokenizer.encode(text,add_special_token=True,max_length=self.max_len+2,truncation=True,return_tensors="pt")
        label_index=[0]+[self.label2index[i] for i in label]+[0]*(self.max_len-len(text)+1)
        label_index=torch.tensor(label_index)

        return text_index.reshape(-1),label_index
    def __len__(self):
        return len(self.texts)

In [36]:
def load_data():
    all_data=pd.read_csv(os.path.join("train_data_public.csv"),encoding="utf-8")
    all_text=all_data["text"].tolist()
    texts=[]
    for text in all_text:
        tmp=[]
        for char in text:
            tmp.append(char)
        texts.append(tmp)
    all_label=all_data["BIO_anno"].tolist()
    labels=[]
    for label in all_label:
        label=label.split(" ")
        labels.append(label)

    label2index={"PAD":0,"UNK":1}
    for label in labels:
        for l in label:
             if l not in label2index:
                 label2index[l] =len(label2index)

    return texts,labels,label2index,list(label2index)


In [37]:
batch_size=10
epoch=100
max_len=100

In [41]:
tokenizer=BertTokenizer.from_pretrained(os.path.join("bert-base-chinese"))
texts,labels,label2index,index2label=load_data()
train_dataset=BertDataset(texts,labels,label2index,max_len,tokenizer)
train_dataloader=DataLoader(train_dataset,batch_size=batch_size,shuffle=False)